# 가설과 추론
앞에서 배운 통계와 확률 이론으로 가설을 검정하고 통계적 추론을 해보자. 데이터와 데이터를 발생시킨 현상에 대한 가설을 세우고 검정을 한다.

## 7.1 통계적 가설검정
가설(hypothesis): "이 동전은 앞뒤가 나올 확률이 공평한 동전이다.", "데이터 과학자는 R보다 Python을 선호한다.", "닫기 버튼이 작아서 찾기 힘든 광고 창을 띄우면 사용자는 해당 사이트를 죽었다 깨어나도 들어가지 않을 것이다." 등과 같은 주장을 의미. \
데이터 통계치에 대한 얘기로 변환될 수 있다. 통계치들은 다양한 가정하에서 특정 분포에 대한 확룰변수의 관측치로 이해할 수 있고, 그런 가정들이 얼마나 타당한지 알 수 있게 해준다. \
고전적인 가설검정에서는 기본 입장을 나타내는 **귀무가설($H_0$, null hypothesis)**과 대비되는 입장을 나타내는 **대립가설($H_1$, alternative hypothesis)** 을 통계적ㅇ로 비교해서 귀무가설을 기각할지 말지 결정한다. 

## 7.2  예시: 동전 던지기
동전이 하나 있다. 이 동전이 공평한 동전인지 아닌지 검정하고 싶다. 이 동전에서 앞면이 나올 확률이 p라고 하면 동전이 공평하다는 의미의 'p = 0.5'이다는 귀무가설이되고 'p $\neq$ 0.5 이다'는 대립가설이 된다. \
동전을 n번 던져 앞면이 나온 횟수 X를 세는 것으로 검정을 진행, 동전 던지기는 각각 베르누이 분포를 다를 것이며, 이는 X가 이항분포를 따르는 확률변수라는 것을 의미한다. 이항분포는 정규 분포로 근사할 수 있다.

In [1]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Binomial(n,p)에 해당되는 mu(평균)와 sigma(표준편차) 계산"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [2]:
def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

확률변수가 정규분포를 따른다는 가정하에 normal_cdf를 사용하면 실제 동전 던지기로부터 얻은 값이 구간 안(혹은 밖)에 존재할 확률을 계산할 수 있다.

In [3]:
# 누적 분포 함수는 확률변수가 특정 값보다 작을 확률을 나타낸다.
normal_probability_below = normal_cdf

# 만약 확률변수가 특정 값보다 작지 않다면, 특정 값보다 크다는 것을 의미한다.
def normal_probability_above(lo: float,
                            mu: float = 0,
                            sigma: float = 1) -> float:
    """mu(평균)와 sigma(표준편차)를 따르는 정규분포가 lo보다 클 확률"""
    return 1 - normal_cdf(lo, mu, sigma)

# 만약 확률변수가 hi보다 작고 lo보다 작지 않다면 확률변수는 hi와 lo 사이에 존재한다.
def normal_probability_between(lo: float,
                              hi: float,
                              mu: float = 0,
                              sigma: float = 1) -> float:
    """mu(평균)와 sigma(표준편차)를 따르는 정규분포가 lo와 hi 사이에 있을 확률"""
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# 만약 확률변수가 범위 밖에 존재한다면 범위 안에 존재하지 않다는 것을 의미한다.
def normal_probability_outside(lo: float,
                              hi: float,
                              mu: float = 0,
                              sigma: float = 1) -> float:
    """mu(평균)와 sigma(표준편차)를 따르는 정규분포가 lo와 hi 사이에 없을 확률"""
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [4]:
def inverse_normal_cdf(p: float,
                      mu: float = 0,
                      sigma: float = 1,
                      tolerance: float = 0.00001) -> float:
    """이진 검색을 사용해 역함수를 근사"""
    # 표준정규분포가 아니라면 표준정규분포로 변환
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z = -10.0  # normal_cdf(-10)은 0에 근접
    hi_z = 10.0  # normal_cdf(10)은 1에 근접
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2  # 중간 값
        mid_p = normal_cdf(mid_z)  # 중간 값의 누적분포 값을 계산
        if mid_p < p:
            low_z = mid_z  # 중간 값이 너무 작다면 더 큰 값들을 검색
        else:
            hi_z = mid_z  # 중간 값이 너무 크다면 더 작은 값들을 검색
    
    return mid_z

반대로, 확률이 주어졌을 때 평균을 중심으로 하는 (대칭적인) 구간을 구할 수도 있다. 예를 들어 분포의 60%를 차지하는 평균 중심의 구간을 구하고 싶다면 양
쪽 꼬리 부분이 각각 분포의 20%를 차지하는 지점을 구하면 된다.

In [5]:
def normal_upper_bound(probability: float,
                      mu: float = 0,
                      sigma: float = 1) -> float:
    """P(Z <= z) = probability인 z 값을 반환"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float,
                           mu: float = 0,
                           sigma: float = 1) -> float:
    """P(Z >= z) = probability인 z 값을 반환"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                           mu: float = 0,
                           sigma: float = 1) -> Tuple[float, float]:
    """
    입력한 probability 값을 포함하고,
    평균을 중심으로 대칭적인 구간을 반환
    """
    tail_probability = (1 - probability) / 2
    
    # 구간의 상한은 tail_probability 값 이상의 확률 값을 갖고 있다.
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # 구간의 하한은 tail_probability 값 이하의 확률 값을 갖고 있다.
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [6]:
# 동전을 1,000번 던져 보자(n = 1000). 가설이 맞다면 X는 대략 평균이 500 이고 표준편차가 15.8인 정규분포를 따를 것이다.
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print("mu_0:", mu_0, "sigma_0:", sigma_0)

mu_0: 500.0 sigma_0: 15.811388300841896


이제 제1종 오류를 얼마나 허용해 줄 것인지를 의미하는 유의수준(significance)을 결정해야 한다. **제 1종 오류**란 비록 $H_0$가 참이지만 $H_0$를 기각하는 'false positive(가양성)' 오류를 의미한다. 유의수준은 보통 5%나 1%로 설정하는 경우가 많은데, 여기서는 유의수준을 5%로 선택해 보자. \
다음의 코드에서 X가 주어진 범위를 벗어나면 귀무가설 $H_0$를 기각하는 가설검정을 고려해 보자.

In [7]:
# (469, 531)
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)  # 유의수준을 5%로 선택함
print(lower_bound, upper_bound)

469.01026640487555 530.9897335951244


p가 정말로 0.5, 즉 $H_0$가 참이라면 X가 주어진 범위를 벗어날 확률은 우리가 원한 대로 5%밖에 되지 않을 것이다. 바꿔 말하면, 만약 $H_0$가 참이라면 이 가설검정은 20번 중 19번은 올바른 결과를 줄 것이다. \
한편 제 2종 오류를 범하지 않을 확률을 구하면 검정력(power)을 알 수 있다. **제 2종 오류**란 $H_0$가 거짓이지만 $H_0$를 기각하지 않는 오류를 의미.\
제 2종 오류를 측정하기 위해 먼저 $H_0$가 거짓이라는 것이 무엇을 의미하는지 알아볼 필요가 있다. (p가 0.5가 아니라는 말은 X의 분포에 관한 많은 것을 알려 주지는 않는다.) 예를 들어, p가 0.55, 즉 동전의 앞면이 나올 확률이 약간 편향되어 있다면 검정력은 다음과 같다.

In [8]:
# p가 0.5라고 가정할 때, 유의수준이 5%인 구간
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# p = 0.55인 경우의 실제 평균과 표준편차
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# 제 2종 오류란 귀무가설(H0)을 기각하지 못한다는 의미
# 즉, X가 주어진 구간 안에 존재할 경우를 의미
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability  # 0.887

print("mu_1:", mu_1, "sigma_1:", sigma_1)
print("type_2_probability:", type_2_probability)
print("power:", power)

mu_1: 550.0 sigma_1: 15.732132722552274
type_2_probability: 0.1134519987046329
power: 0.886548001295367


한편 $p \leq 0.5$, 즉 동전이 앞면에 편향되지 않을 경우를 귀무가설로 정한다면 X가 500보다 크면 귀무가설을 기각하고, 500보다 작다면 기각하지 않는 단측검정 (one-sided test)이 필요해진다. 유의수준이 5%인 가설검정을 위해서는 normal_probability_below를 사용하여 분포의 95%가 해당 값 이하인 경계 값을 찾을 수 있다.|

In [10]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
print(hi)  # (< 531, 분포 상위 부분에 더 높은 확률을 주기 위해서)

526.0073585242053


In [11]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print("type_2_probability:", type_2_probability)
print("power:", power)

type_2_probability: 0.06362051966928273
power: 0.9363794803307173


이 가설검정은 더 이상 X가 469보다 작을 때 $H_0$를 기각하는게 아니라($H_1$이 참이라면 이는 거의 발생하지 않을 것이다.) X가 526 ~ 531 사이일때 $H_0$를 기각하기 때문에 ($H_1$이 참이라면 이는 가끔 발생할 것이다.) 전보다 검정력이 좋아졌다고 볼 수 있다.

## 7.3 p-value
가설을 바라보는 다른 관점은 $p$-value이다. 이는 어떤 확률값을 기준으로 구간을 선택하는 대신에, $H_0$가 참이라고 가정하고 실제로 관측된 값보다 더 극단적인 값이 나올 확률을 구하는 것이다. \
동전이 공평한지 확인해보기 위해 양측검정을 해보자

In [12]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    """
    mu(평균)와 sigma(표준편차)를 따르는 정규분포에서 x같이
    극단적인 값이 나올 확률은 얼마나 될까?
    """
    if x >= mu:
        # 만약 x가 평균보다 크다면 x보다 큰 부분이 꼬리다.
        return 2 * normal_probability_above(x, mu, sigma)
    
    else:
        # 만약 x가 평균보다 작다면 x보다 작은 부분이 꼬리다.
        return 2  * normal_probability_below(x, mu, sigma)

In [13]:
# 동전의 앞면이 나온 경우가 530번 관측되었다면  P-value는 다음과 같다.
two_sided_p_value(529.5, mu_0, sigma_0)  # 0.062

0.06207721579598835

In [19]:
# 시뮬레이션을 해보면 우리의 추정갑싱 그럴듯하다는 것을 확인할 수 있다.
import random

extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(1000))  # 앞면이 나온 경우를 세어본다. 동전 1000번을 던져서
    if num_heads >= 530 or num_heads <= 470:  # 그리고 극한 값이
        extreme_value_count += 1  # 몇 번 나오는지 세어 본다.

# p-value was 0.062 => ~62 extreme values out of 1000
assert 59 < extreme_value_count < 65, f"{extreme_value_count}"

계산된 p-value가 5%보다 크기 때문에 귀무가설을 기각하지 않는다. 만약 동전의 앞면이 532번 나왔다면 p-value는 5%보다 작을 것이고, 이 경우에는 귀무가설을 기각할 것이다.

In [20]:
two_sided_p_value(531.5, mu_0, sigma_0)  # 0.0463

0.046345287837786575

이전 가설검정에 비해 통계를 보는 관점만 다를 뿐 동일한 검정 방법이다. 같은 방식으로

In [21]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

동전의 앞면이 525번 나왔다면 단측검정을 위한 p-value는 다음과 같이 계산되며

In [23]:
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582072

귀무가설을 기각하지 않을 것이다. 만약 동전의 앞면이 527번 나왔다면 p-value는 다음과 같이 계산되며 귀무가서을 기각할 것이다.

In [24]:
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

## 7.4 신뢰구간
앞에서 동전을 던져 앞면이 나올 확률을 p, p 값은 앞면이 나올 미지의 분포를 나타내는 파라미터였다. **만약 사건에 대한 분포를 모른다면 관측된 값에 대한 신뢰구간(confidence interval)을 사용하여 가설을 검정할 수 있다.** \
예를 들어 공평하지 않은 동전에 대한 확률을 앞면이면 1, 뒷면이면 0인 베르누이 확률변수의 평균을 이용하여 추정할 수 있다. 만약 동전을 1,000번 던져서 앞면이 525번 나왔다면 p는 0.525로 추정할 수 있다. \
이 추정값에 대해 얼마나 신뢰할 수 있을까? 만약 p의 정확한 값을 알고 있다면 중심극한 정리를 사용하여 베르누이 확률변수들의 평균은 대략 평균이 p이고 표준편차가 다음과 같은 정규분포 로 추정할 수 있을 것이다.

math.sqrt(p * (1 - p) / 1000) \
p의 정확한 값을 모른다는 추정값을 사용할 수 있다.

In [26]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)  # 0.0158

이렇게 계산하는 것이 완벽하게 옳은 방법은 아니지만, 많은 사람이 이렇게 계산한다. 정규분포의 근사를 사용하면 진짜 p가 다음 구간 안에 포함되어 있을 것이라고 95% 확신할 수 있다.

In [28]:
normal_two_sided_bounds(0.95, mu, sigma)

#  이 경우 0.5는 신뢰구간 안에 있기 때문에 동전은 공평하지 않다고 결론을 내릴 수 가 없다. 반대로 앞면이 540번 나왔다면

(0.4940490278129096, 0.5559509721870904)

In [30]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)  # 0.0158
normal_two_sided_bounds(0.95, mu, sigma)

# '공평한 동전'에 대한 확률은 계산된 신뢰구간 밖에 존재하게 된다. 공평한 동전이라는 가설은, 가설이 참이라면 모든 경우 95%에 대해 가설검정을
# 통과하지 못하게 된다.

(0.5091095927295919, 0.5708904072704082)

## 7.5 p 해킹
귀무가설을 잘못 기각하는 경우가 5%인 가설검정은 정의에서 알 수 있듯이 모든 경우의 5%에서 귀무가설을 잘못 기각한다.

In [32]:
from typing import List

def run_experiment() -> List[bool]:
    """동전을 1000번 던져서 True = 앞면, False = 뒷면"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """신뢰구간을 5%로 설정"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

assert num_rejections == 46

의미 있는 결과를 찾으려고 노력하다면 보통 의미 있는 결과를 찾을 수 있다는 것을 의미한다. 주어진 데이터에 다양한 가설을 검정하다 보면 이 중 하나는 반드시 의미 있는 가설로 보일 것이다. 적절한 이상치를 제거하면 0.05이하의 p-value를 구 할 수 있다.
이렇게 p-value의 관점에서 추론을 하면 'p 해킹'이 발생할 수 있다. 

#### 데이터 과학을 제대로 하기 위한 3가지
1. 가설은 데이터를 보기 전에 세운다. 
2. 데이터를 전처리할 때는 세워둔 가설을 잠시 잊는다. 
3. p-value가 전부는 아니다. (대안으로 베이즈 추론을 사용할 수 있다.)

## 7.6 예시: A/B test 해보기
사용자에게 임의로 두 개의 광고 중 하나를 보여 주고 실제로 광고를 클릭하는 사용자의 수를 살펴보는 실험을 진행 \
만약 광고 A, B가 있을 때 둘의 차이가 명확하지 않다면 어떻게 해야 할까? 통계적 추론은 이럴때 사용해야 한다. \ 
$N_A$명의 사용자가 광고 A를 보았고, 그중 $n_A$명이 광고를 클릭했다고 해보자. 각 사용자가 광고를 클릭하는 것은 베르누이 시행으로 볼 수 있으며 각 사용자가 광고 A를 클릭할 확률을 $p_A$f라고 정의하자. \
$\Rightarrow$ $n_A/N_A$는 평균이 $p_A$이고 표준편차가 $\sigma_A = \sqrt{p_A(1-p_A)/N_A}$ 인 정규분포에 근접하다는 것을 알 수 있다. \
$\Rightarrow$ $n_B/N_B$는 평균이 $p_B$이고 표준편차가 $\sigma_B = \sqrt{p_B(1-p_B)/N_B}$ 인 정규분포에 근접하다는 것을 알 수 있다.

In [33]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

만약 두 정규분포가 독립이라면(각각의 베르누이 시행은 독립적인 시행이기 때문에 적절한 가정이다), 두 정규분포의 차이 또한 평균이 $p_B - p_A$이고 표준편차가 $\sqrt{{\sigma_A}^2 + {\sigma_B}^2}$인 정규분포를 따른다.

In [34]:
def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [36]:
# 1000명 중에 200명이 '맛이 좋다'라는 광고를 클릭했고 1000명 중 180명이 '편향이 없어요'라는 광고를 클릭했다.
z = a_b_test_statistic(1000, 200, 1000, 180)
z

-1.1403464899034472

In [37]:
two_sided_p_value(z)

0.2541419765422359

두 값이 다르기 때문에 두 분포가 다르다고 결론을 내릴 수가 없다. 만약 150명이 '편향이 없어요'라는 광고를 클릭했다면 다음과 같은 결론이 나온다.

In [39]:
z = a_b_test_statistic(1000, 200, 1000, 150)
z

-2.948839123097944

In [40]:
two_sided_p_value(z)

0.003189699706216853

두 광고가 동일하게 효과적이라면 이렇게 큰 차이가 발생할 확률은 불과 0.003이라는 것을 의미한다.

## 7.7 베이즈 추론
알려지지 않은 파라미터를 확루변수로 보는 방법이 있다. 분석가에게 파라미터에 대한 사전분포가 주어지고, 관측된 데이터와 베이즈 정리를 사용하여 사후분포를 갱신할 수 있다. 통계적 검정에 대해 확률적으로 결론을 내는 대신에 파라미터에 대해 확률적으로 결론을 낼 수 있다. \
예를 들어 동전 던지기 예시처럼 알려지지 않은 파라미터가 확률이라고 해보자. 보통은 모든 확률 값이 0 ~ 1 사이에서 정의되는 **베타분포**를 사전분포로 사용한다.

In [ ]:
def 